In [1]:
import pandas as pd
import numpy as np 

## read rdu attr sheet

In [2]:
rdu_df = pd.read_excel('C:/Users/liuleo/Documents/DataManagement/weightedtree/weightedtree/data/RDU_master.xlsx',
                      sheet_name='RDU_Attributes')

In [3]:
rdu_df.head()

,Attribute ID,Information entity ID,Information entity,Attribute group ID,Attribute group,Attribute Name,Number of Characters,Last Modified by,Attribute definition,Variable type,Aggregation time interval,Analytical record
0,A73,IE2,CLAIM,AG5,ACCIDENT DETAILS,ACCIDENT_DESCRIPTION,20,NaN,DESCRIPTION OF THE ACCIDENT.,ATTRIBUTE,CUT-OFF,Claim
1,A74,IE2,CLAIM,AG5,ACCIDENT DETAILS,ACCIDENT_LOCATION,17,NaN,PLACE OF AN ACCIDENT/CONVICTION.,ATTRIBUTE,CUT-OFF,Claim
2,A75,IE2,CLAIM,AG5,ACCIDENT DETAILS,BODILY_INJURY_INDICATOR,23,NaN,INDICATES WHETHER THE INCIDENT LED TO ANY BODI...,ATTRIBUTE,CUT-OFF,Claim
3,A76,IE2,CLAIM,AG5,ACCIDENT DETAILS,DEATH_INDICATOR,15,NaN,INDICATES WHETHER THE INCIDENT LED TO ANY DEATHS.,ATTRIBUTE,CUT-OFF,Claim
4,A77,IE2,CLAIM,AG5,ACCIDENT DETAILS,BODYSHOP_INDICATOR,18,NaN,UNIQUE IDENTIFIER OF GARAGE / BODYSHOP WHERE R...,ATTRIBUTE,CUT-OFF,Claim


In [4]:
print rdu_df.shape
print rdu_df['Attribute Name'].nunique()

(1603, 12)
1463


In [5]:
rdu_df = rdu_df[['Attribute ID','Attribute Name','Information entity','Attribute group','Attribute definition']]

## read rdu viz csv

In [6]:
viz_df = pd.read_csv('C:/Users/liuleo/Documents/DataManagement/weightedtree/weightedtree/data/test2.csv')
print viz_df.shape
print viz_df['Level4'].nunique()

(1603, 12)
1462


In [7]:
viz_df_full = viz_df.merge(rdu_df,how='inner',left_on=['Level4','Level2','Level3','Comments4'], 
                           right_on=['Attribute Name','Information entity','Attribute group','Attribute definition'])

In [8]:
viz_df_full.drop(['Attribute Name','Information entity','Attribute group'], inplace=True, axis=1)

In [9]:
viz_df_full.shape

(1602, 14)

In [10]:
test = viz_df_full.groupby(['Level4','Level3','Level2'], as_index=False).agg({'All':pd.Series.count})

In [15]:
viz_df_full

(1602, 14)

## Reading usecase sheet 

In [28]:
excel_df = pd.ExcelFile("C:/Users/liuleo/Documents/DataManagement/weightedtree/weightedtree/data/RDU_master.xlsx")

excel_df.sheet_names

uc_df = excel_df.parse('Mapping')

In [34]:
uc_df.columns.values

array([u'Use case ID', u'Use case description', u'Attribute ID',
       u'Attribute name', u'Product Category', u'Product line', u'Product',
       u'Relevance', u'Channel'], dtype=object)

In [43]:
# join for lead aquisition : UC1.0 Increase Customer acquisition Identify most profitable leads/prospects to acquire through "People like you" analysis in the bancassurance channel
uc_df_xsell = uc_df[uc_df['Use case ID']=='UC1.0'][['Attribute ID','Use case ID']].reset_index(drop=True)

In [45]:
viz_df_full = viz_df_full.merge(uc_df_xsell,how='left',left_on='Attribute ID',right_on='Attribute ID')

viz_df_full = viz_df_full.rename(columns={'Use case ID':'Marketing'})

In [59]:
# join for u18 : Explore customer relationships to identify fraud patterns in social networks  based on internal and social media data
uc_df_fraud = uc_df[uc_df['Use case ID']=='UC18.0'][['Attribute ID','Use case ID']].reset_index(drop=True)

In [60]:
viz_df_full = viz_df_full.merge(uc_df_fraud,how='left',left_on='Attribute ID',right_on='Attribute ID')

viz_df_full = viz_df_full.rename(columns={'Use case ID':'Claim'})

In [67]:
UC_COLS = ['Marketing','Claim']
for col in UC_COLS:
    viz_df_full[col] = viz_df_full[col].apply(lambda x : 0 if pd.isnull(x) else 1)

In [70]:
viz_df_full.to_csv('C:/Users/liuleo/Documents/DataManagement/weightedtree/weightedtree/data/test2_uc.csv',index=False)